# Predict Survival of the Titanic Passengers

Dataset Source: [https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html](https://ww2.amstat.org/publications/jse/v3n3/datasets.dawson.html)

## Table of Contents
- [Load Libraries](#load_libraries)
- [Access data](#access_data)
- [Split Data into Training and Test set](#training_test)
- [Build Logistic Regression Model](#build_model)
- [Predict for Test data](#test_data)
- [Evaluate the Model](#evaluate_model)

<a id="load_libraries"></a>
## Load Libraries

The Spark and Python libraries that you need are preinstalled in the notebook environment and only need to be loaded.

Run the following cell to load the libraries you will work with in this notebook:

In [ ]:
# PySpark Machine Learning Library
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row, SQLContext

import os
import sys
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Library for confusion matrix, precision, test error
from pyspark.mllib.evaluation import MulticlassMetrics
# Library For Area under ROC curve and Area under precision-recall curve
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Assign resources to the application
sqlContext = SQLContext(sc)

In [ ]:
# The data will be loaded into an array.
# This is the summary of the data structure, including the column position and name.
# The first filed starts from position 0. 

# 0 Interaction ID - This is a VarChar identifaction that serves to make each entry unique and accesible
# 1 Article_URL - This is the URL where the text was taken from
# 2 Content - The text of each tweet
# 3 Time - timestamp of content
# 4 Relevant - Description of whether or not the tweet was about the subject matter
# 5 Sentiment - 5 different sentiments expressed as to the persons feelings
# Gender - male/female description of subject
# Dizziness - yes/no description of whether or not dizziness occurred
# Convulsions - yes/no description of whether or not convulsions were present in the patient
# HEART PALPITATIONS - yes/no description of whether or not heart palpitations were present
# SHORTNESS OF BREATHE - yes/no description of the whether or not the person experienced breathing difficulties
# HEADACHES - yes/no description of headaches being present 
# DRUG EFFECT DECREASED - yes/no description of whether or not the effect of claritin decreased over time
# ALLERGIES WORSE AFTER TAKING DRUG - yes/no description of symptoms getting worse after medication
# BAD INTERACTION BETWEEN CLARITIN AND ANOTHER DRUG. - yes/no description of synergistic reaction with other drug
# NAUSEA (MADE THE PERSON NAUSEOUS) - yes/no description of whether the person was nauseous or not
# CAUSED INSOMNIA (THE PERSON WASNT ABLE TO SLEEP) - yes/no description of whether the person was able to sleep

# Label is a target variable. PersonInfo is a list of independent variables besides unique identifier

LabeledDocument = Row("Interaction ID", "Article URL", "Content", " Time", "Relevant", "Sentiment", "Gender", "Dizziness", "Convulsions", "Heart Palpitations", "Shortness of Breathe", "Headaches", "Drug Effect Decreased", "Allergies Worse After Taking the Drug", "Bad Interaction Between Claritin and Another Drug", "Nausea", "Caused Insomnia")

# Define a function that parses the raw CSV file and returns an object of type LabeledDocument

def parseDocument(line):
    values = [str(x) for x in line.split(',')] 
    if (values[4]>'0'):
      Survived = 1.0
    else:
      Survived = 0.0
        
    textValue = str(values[1]) + " " + str(values[2])+" " + str(values[3])
    return LabeledDocument(values[5], textValue, Survived)

<a id="access_data"></a>
## Access Data
Before you can access data in the data file in the Object Storage, you must setup the Spark configuration with your Object Storage credentials. 

To do this, click on the cell below and select the **Insert to code > Insert Spark Session DataFrame** function from the Files tab below the data file you want to work with.

<div class="alert alert-block alert-info">The following code contains the credentials for a file in your IBM Cloud Object Storage. Delete the code starting from `from pyspark.sql import SparkSession` line before you run the cell.</div>

In [ ]:
# Object Storage Credentials


Now let's load the data into a `Spark RDD` and output the number of rows and first 5 rows.
Each project you create has a bucket in your object storage. You can get the bucket name from the project Settings page. Change the string `BUCKET` to the bucket name

In [ ]:
data = sc.textFile(cos.url('Titanic.csv', 'BUCKET'))
print "Total records in the data set:", data.count()
print "The first 5 rows"
data.take(5)

Crate DataFrame from RDD

In [ ]:
#Load the data into a ataframe, parse it using the function above
documents = data.filter(lambda s: "Name" not in s).map(parseDocument)
TitanicData = documents.toDF() # ToDataFrame
print "Number of records: " + str(TitanicData.count())
print "First 5 records: "
TitanicData.take(5)

<a id="training_test"></a>
## Split Data into Training and Test set

We divide the data into training and test set.  The training set is used to build the model to be used on future data, and the test set is used to evaluate the model.

In [ ]:
# Divide the data into training and test set
(train, test) = TitanicData.randomSplit([0.8, 0.2])
print "Number of records in the training set: " + str(train.count())
print "Number of records in the test set: " + str(test.count())
# Output first 20 records in the training set
print "First 20 records in the training set: "
train.show()

<a id="build_model"></a>
## Build Logistic Regression Model

We use the Pipeline of SparkML to build the Logistic Regression Model

In [ ]:
# set up Logistic Regression using Pipeline of SparkML
tokenizer = Tokenizer(inputCol="PersonInfo", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [ ]:
# set up Logistic Regression Model
# the stages are executed in order
model = pipeline.fit(train)
#[stage.coefficients for stage in model.stages if hasattr(stage, "coefficients")]
# model.stages[2].intercept
model.stages[2].coefficients

<a id="test_data"></a>
## Predict for Test data

In [ ]:
# Make predictions on test documents and print columns of interest
prediction = model.transform(test)
selected = prediction.select("PersonInfo", "prediction", "probability")
for row in selected.collect():
    print row
#for row in prediction.collect():
#    print row

In [ ]:
#Tabulate the predicted outcome
prediction.select("prediction").groupBy("prediction").count().show(truncate=False)

In [ ]:
#Tabulate the actual outcome
prediction.select("label").groupBy("label").count().show(truncate=False)

In [ ]:
# This table shows:
# 1. The number of survived passengers predicted as survived
# 2. The number of survived passengers predicted as not survived
# 3. The number of not survived passengers predicted as survived
# 4. The number of not survived passengers predicted as not survived

prediction.crosstab('label', 'prediction').show()

<a id="evaluate_model"></a>
## Evaluate the Model

We evaluate the model on a training set and on a test set.  The purpose is to measure the model's predictive accuracy, including the accuracy for new data.

In [ ]:
# Evaluate the Logistic Regression model on a training set
# Select (prediction, true label) and compute test error
pred_lr=model.transform(train).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Test Accuracy = %s" %accuracy_lr)
print ("Test Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

In [ ]:
# Evaluate the Logistic Regression model on a test set
# Select (prediction, true label) and compute test error
pred_lr=model.transform(test).select("prediction", "label")
eval_lr=MulticlassClassificationEvaluator (
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr=eval_lr.evaluate(pred_lr)
# create RDD
predictionAndLabels_lr=pred_lr.rdd
metrics_lr=MulticlassMetrics(predictionAndLabels_lr)
precision_lr=metrics_lr.precision(1.0)
recall_lr=metrics_lr.recall(1.0)
f1Measure_lr = metrics_lr.fMeasure(1.0, 1.0)
print("F1 Measure = %s" % f1Measure_lr)
print ("Test Accuracy = %s" %accuracy_lr)
print ("Test Error = %s" % (1-accuracy_lr))
print ("Precision = %s" %precision_lr)
print ("Recall = %s" %recall_lr)

In [ ]:
bin_lr=BinaryClassificationMetrics(predictionAndLabels_lr)

# Area under precision-recall curve
print("Area under PR = %s" % bin_lr.areaUnderPR)
# Area under precision-recall curve
print("Area under ROC = %s" % bin_lr.areaUnderROC)